<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 4 Positions </span> </p>

bla

# Setup

In [0]:
//#!import "//ifrs17/v1.1.1/CalculationEngine"
#!import "../../../ifrs17/CalculationEngine"

# Positions

In [0]:
public interface IWithGetValueFromValues : IScope<ImportIdentity, ImportStorage>
{
    private int shift => GetStorage().GetShift(0);
    private int timeStep => GetStorage().GetTimeStep(0);
    
    public double GetValueFromValues(double[] Values) => GetStorage().GetValuationPeriod(Identity) switch {
        ValuationPeriod.BeginningOfPeriod => Values.ElementAtOrDefault(shift),
        ValuationPeriod.MidOfPeriod => Values.ElementAtOrDefault(shift + Convert.ToInt32(Math.Round(timeStep / 2d, MidpointRounding.AwayFromZero)) - 1),
        ValuationPeriod.Delta => Values.Skip(shift).Take(timeStep).Sum(),
        ValuationPeriod.EndOfPeriod  => Values.ElementAtOrDefault(shift + timeStep),
        ValuationPeriod.NotApplicable => default };
}

In [0]:
public interface NominalTechnicalMargin : IScope<ImportIdentity, ImportStorage>, IWithGetValueFromValues
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<NominalTechnicalMargin>(s => s.WithApplicability<NominalTechnicalMarginDefaultValue>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I)
                                                       .WithApplicability<NominalTechnicalMarginDefaultValue>(x => x.Identity.AocType == AocTypes.CF || x.Identity.AocType == AocTypes.IA ||
                                                                                                                   x.Identity.AocType == AocTypes.EA || x.Identity.AocType == AocTypes.AM ));
    
    [NotVisible] string EconomicBasis => EconomicBases.L;

    double Value => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes.Except(GetStorage().GetNonAttributableAmountType())
                        .Sum(at => GetValueFromValues(GetScope<NominalCashflow>((Identity, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Values)) 
                        + GetScope<RaLocked>(Identity).Value;

    double AggregatedValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                .Sum(aoc => GetScope<NominalTechnicalMargin>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
}

public interface NominalTechnicalMarginDefaultValue : NominalTechnicalMargin
{
    double NominalTechnicalMargin.Value => default;
}

In [0]:
Import.DefineFormat("IFRS 4 Cashflow", async (options, dataSet) => {
    Activity.Start();
    var args = await GetArgsAndCommitPartitionAsync<PartitionByReportingNodeAndPeriod>(dataSet, options.TargetDataSource) with {ImportFormat = ImportFormats.Cashflow};
    if(Activity.HasErrors()) return Activity.Finish();

    await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, args, options.TargetDataSource);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var workspace = Workspace.CreateNew();
    var log = await ParseCashflowsToWorkspaceAsync(dataSet, args, workspace, options.TargetDataSource);
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    var workspaceToCompute = Workspace.CreateNew();
    workspaceToCompute.Initialize(x => x.FromSource(options.TargetDataSource));
    var storage = new ImportStorage(args, workspaceToCompute, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish().Merge(log);

    var universe = Scopes.ForStorage(storage).ToScope<IModel>();
    var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
    var ivs = universe.GetScopes<NominalTechnicalMargin>(identities).Select(x => new IfrsVariable{ EstimateType = "IFRS4EQ",
                                                                                                   DataNode = x.Identity.DataNode,
                                                                                                   AocType = x.Identity.AocType,
                                                                                                   Novelty = x.Identity.Novelty,
                                                                                                   Value = x.Value,
                                                                                                   Partition = storage.TargetPartition });
    if(Activity.HasErrors()) return Activity.Finish().Merge(log);

    workspace.Reset(x => x.ResetType<IfrsVariable>());
    await workspace.UpdateAsync<IfrsVariable>(ivs.Where(x => storage.DefaultPartition != storage.TargetPartition || Math.Abs(x.Value) >= Precision));
    await workspace.CommitToAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(workspaceToCompute, storage.TargetPartition, snapshot : false);
        //filter : x => storage.EstimateTypesByImportFormat[args.ImportFormat].Contains(x.EstimateType) && storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    await workspaceToCompute.CommitToTargetAsync(options.TargetDataSource);  // TODO: substitute workspaceToCompute with options.TargetDataSource
    return Activity.Finish().Merge(log); 
});